In [17]:
import os
import pandas as pd
import re
import datetime
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


# Function to list all chat files in the directory structure
def list_chat_files(date_directory):
    chat_files = []
    for date_folder in os.listdir(date_directory):
        date_path = os.path.join(date_directory, date_folder)
        if os.path.isdir(date_path):
            for team_folder in os.listdir(date_path):
                team_path = os.path.join(date_path, team_folder)
                if os.path.isdir(team_path):
                    for person_folder in os.listdir(team_path):
                        person_path = os.path.join(team_path, person_folder)
                        if os.path.isdir(person_path):
                            for file in os.listdir(person_path):
                                if file.endswith('.txt'):
                                    chat_files.append(os.path.join(person_path, file))
    return chat_files

def parse_chat_file_for_delay_analysis(file_path):
    # Extract person_name and date from the file path
    person_name = os.path.basename(os.path.dirname(file_path))
    folder_date_str = os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(file_path))))
    expected_date_minus_one = pd.to_datetime(folder_date_str, format='%Y-%m-%d').date() - datetime.timedelta(days=1)

    delay_messages = []

    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    last_person_msg_time = None
    last_person_msg_index = None

    for i, line in enumerate(lines):
        message_match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2} [ap]m) - (.*?): (.*)', line)
        if message_match:
            date_time_str, sender, message = message_match.groups()
            date_time = pd.to_datetime(date_time_str, format='%d/%m/%y, %I:%M %p')

            if date_time.date() != expected_date_minus_one:
                continue

            # Check if the sender is a person (not purely numeric or starting with +)
            is_person = sender is not None and not (sender.strip().isnumeric() or sender.startswith('+'))

            logging.info(f"Sender: {sender}, Is Person: {is_person}, Message: {message}")  # Debug line

            if is_person:
                if last_person_msg_time is not None:
                    # Calculate the time difference in minutes
                    time_diff_seconds = (date_time - last_person_msg_time).total_seconds()
                    time_diff_minutes = time_diff_seconds / 60
                    logging.info(f"Time Difference (minutes): {time_diff_minutes}")  # Debug line
                    if time_diff_minutes > 15:
                        # There is a delay
                        delay = True
                    else:
                        # No delay
                        delay = False
                        time_diff_minutes = None
                else:
                    # No previous message from a person
                    delay = False
                    time_diff_minutes = None
                last_person_msg_time = date_time
                last_person_msg_index = i
            else:
                # Not a message from a person
                delay = False
                time_diff_minutes = None

            if delay:
                # Fetch reference messages
                start_index = max(last_person_msg_index - 3, 0)
                end_index = min(i + 4, len(lines) - 1)
                reference_msgs = lines[start_index:end_index + 1]

                delay_messages.append((date_time, sender, message, delay, time_diff_minutes, reference_msgs))

    return delay_messages, file_path

            
def process_all_files(chat_files):
    all_delay_messages = []

    for file_path in chat_files:
        logging.debug(f"Processing file: {file_path}")  # Debug line
        delay_messages, file_path_from_function = parse_chat_file_for_delay_analysis(file_path)
        for message in delay_messages:
            all_delay_messages.append((*message, file_path_from_function))

    return all_delay_messages



def create_delay_data_dataframe(all_delay_data):
    columns = ['Date', 'Person', 'Chat Group Name', 'Delay', 'Delay in Mins', 'Reference Messages']
    df_data = []

    for delay_instance in all_delay_data:
        date_time, sender, message, delay, time_diff_minutes, reference_msgs, file_path = delay_instance

        file_name = os.path.basename(file_path)
        logging.info(f"Processing file name: {file_name}")  # Debugging line

        # Regex to extract the chat group name
        group_name_match = re.match(r'WhatsApp Chat with (.+?)\.txt', file_name)
        chat_group_name = group_name_match.group(1) if group_name_match else "Unknown Group"

        df_data.append([date_time, sender, chat_group_name, delay, time_diff_minutes, reference_msgs])

    delay_df = pd.DataFrame(df_data, columns=columns)
    return delay_df



# Main Execution
date_directory = "C:\\Users\\mauriceyeng\\Python\\Daily-Reports\\Test\\filtered_chats"
chat_files = list_chat_files(date_directory)
all_delay_data = process_all_files(chat_files)
delay_analysis_df = create_delay_data_dataframe(all_delay_data)

# Display the first few rows of the DataFrame
print(delay_analysis_df.head())


2023-11-29 16:30:41,610 - INFO - Sender: Arshita, Is Person: True, Message: 11/20/2023 11:33:18	Abdulrazak Abdullahi 	abdulrazak2abdul@gmail.com	+234906443753	3/27/2001	Male	Al-ihsan mallawa academy 	Private	Nigeria 	Abdullahi shuaibu 	+2347033854402	civil servant 	Aishatu Abdullahi 	+2349036605586	Housewife 	Bachelors Course	chemical engineering 	2019	2021	Yes	Close Friend or Relative.
2023-11-29 16:30:41,611 - INFO - Sender: Arshita, Is Person: True, Message: Source: Nirnoy EWYL23E0617
2023-11-29 16:30:41,611 - INFO - Time Difference (minutes): 0.0
2023-11-29 16:30:41,611 - INFO - Sender: Arshita, Is Person: True, Message: Add child
2023-11-29 16:30:41,612 - INFO - Time Difference (minutes): 100.0
2023-11-29 16:30:41,613 - INFO - Sender: Arshita, Is Person: True, Message: Greetings ! 
2023-11-29 16:30:41,613 - INFO - Time Difference (minutes): 11.0
2023-11-29 16:30:41,614 - INFO - Sender: Arshita, Is Person: True, Message: <Media omitted>
2023-11-29 16:30:41,614 - INFO - Time Differe

2023-11-29 16:30:41,724 - INFO - Time Difference (minutes): 64.0
2023-11-29 16:30:41,724 - INFO - Sender: Arshita, Is Person: True, Message: Let us connect at  *11:30 pm CAT on Tuesday(28/11/23) to resume your session (In case you have any challenges or lessons, you can let me know and we will plan accordingly)*
2023-11-29 16:30:41,724 - INFO - Time Difference (minutes): 266.0
2023-11-29 16:30:41,724 - INFO - Sender: Arshita, Is Person: True, Message: <Media omitted>
2023-11-29 16:30:41,724 - INFO - Time Difference (minutes): 0.0
2023-11-29 16:30:41,732 - INFO - Sender: Arshita, Is Person: True, Message: Greetings!
2023-11-29 16:30:41,732 - INFO - Sender: +263 78 599 2436, Is Person: False, Message: Yes
2023-11-29 16:30:41,732 - INFO - Sender: Arshita, Is Person: True, Message: Greetings! 
2023-11-29 16:30:41,732 - INFO - Time Difference (minutes): 14.0
2023-11-29 16:30:41,732 - INFO - Sender: +263 78 599 2436, Is Person: False, Message: Im fine
2023-11-29 16:30:41,738 - INFO - Sender:

2023-11-29 16:30:41,808 - INFO - Time Difference (minutes): 0.0
2023-11-29 16:30:41,835 - INFO - Sender: Arshita, Is Person: True, Message: Alright , we have come to the end of your counselling session, now before I guide you for Step 2 which is the Eligibility Check.
2023-11-29 16:30:41,835 - INFO - Sender: Arshita, Is Person: True, Message: To learn more about the type of students we look for, please go through these videos in which Edoofians from your country have shared their experience
2023-11-29 16:30:41,842 - INFO - Time Difference (minutes): 0.0
2023-11-29 16:30:41,842 - INFO - Sender: Arshita, Is Person: True, Message: For any question apart from whatever has been discussed, I would appreciate it if you refer to the FAQ document which has answers to most of the queries. Furthermore, if you have any other query I would suggest that you note down all of the questions and get back to me
2023-11-29 16:30:41,842 - INFO - Time Difference (minutes): 0.0
2023-11-29 16:30:41,842 - INFO

2023-11-29 16:30:41,932 - INFO - Time Difference (minutes): 8.0
2023-11-29 16:30:41,933 - INFO - Sender: +263 78 733 4360, Is Person: False, Message: I see myself helping patients who have undergone surgery or have serious medical conditions that require rehabilitation as a physiotherapist 
2023-11-29 16:30:41,933 - INFO - Sender: Arshita, Is Person: True, Message: I am glad that you are willing to work hard for your career.I believe you will be researching more about the course in detail.
2023-11-29 16:30:41,934 - INFO - Time Difference (minutes): 4.0
2023-11-29 16:30:41,934 - INFO - Sender: +263 78 733 4360, Is Person: False, Message: From what i have researched about the Indian Education System I have learnt that it offers top-notch education especially in the field of science because of innovation, research, and technology. Also the Indian educational system is more accessible, adaptable, and practical as a result of technology. I have also learnt that its technology-based educatio

2023-11-29 16:30:41,967 - INFO - Time Difference (minutes): 0.0
2023-11-29 16:30:41,967 - INFO - Sender: Arshita, Is Person: True, Message: The EWYL Program, therefore, is going to help in the overall growth of a student and help them provide a boost in keeping up with the changing market strategies, needs, and tactics. Promising a better future for its students, and converting an Individual into a Market Ready Professional. Who would have every weapon in their arsenal to conquer the place they shall be landing their foot in.
2023-11-29 16:30:41,967 - INFO - Time Difference (minutes): 0.0
2023-11-29 16:30:41,967 - INFO - Sender: Arshita, Is Person: True, Message: *Projects with Brands and Companies:* -
2023-11-29 16:30:41,967 - INFO - Time Difference (minutes): 0.0
2023-11-29 16:30:41,967 - INFO - Sender: Arshita, Is Person: True, Message: *Recruitment Partners:* -
2023-11-29 16:30:41,967 - INFO - Time Difference (minutes): 0.0
2023-11-29 16:30:41,967 - INFO - Sender: Arshita, Is Perso

...

2023-11-29 16:30:57,782 - INFO - Processing file name: WhatsApp Chat with Kabongwe K Sibanda Edoofa (11_10) SZ.txt
2023-11-29 16:30:57,782 - INFO - Processing file name: WhatsApp Chat with Kayembe Israel Edoofa(30_10)SR300.txt
2023-11-29 16:30:57,782 - INFO - Processing file name: WhatsApp Chat with Kelvin Manzungu Edoofa (10_10) SZ.txt
2023-11-29 16:30:57,782 - INFO - Processing file name: WhatsApp Chat with Kudakwashe_Edoofa(20_04)HP.txt
2023-11-29 16:30:57,782 - INFO - Processing file name: WhatsApp Chat with Kumbirai Kelvin Tsapo  EDOOFA(01_11)SR.txt
2023-11-29 16:30:57,782 - INFO - Processing file name: WhatsApp Chat with Kumbirai Kelvin Tsapo  EDOOFA(01_11)SR.txt
2023-11-29 16:30:57,782 - INFO - Processing file name: WhatsApp Chat with Kundai Kamwaza EDOOFA(15_11)SR.txt
2023-11-29 16:30:57,782 - INFO - Processing file name: WhatsApp Chat with Kuzoraunye Tatenda EDOOFA(15_11)SR 2025.txt
2023-11-29 16:30:57,782 - INFO - Processing file name: WhatsApp Chat with Lee Makore EDOOFA(15_

2023-11-29 16:30:57,816 - INFO - Processing file name: WhatsApp Chat with Zhylani_Edoofa(22_11)HP.txt
2023-11-29 16:30:57,816 - INFO - Processing file name: WhatsApp Chat with Zhylani_Edoofa(22_11)HP.txt
2023-11-29 16:30:57,816 - INFO - Processing file name: WhatsApp Chat with Albert Phiri Edoofa (16_11) SZ.txt
2023-11-29 16:30:57,816 - INFO - Processing file name: WhatsApp Chat with Albert Phiri Edoofa (16_11) SZ.txt
2023-11-29 16:30:57,816 - INFO - Processing file name: WhatsApp Chat with Albert Phiri Edoofa (16_11) SZ.txt
2023-11-29 16:30:57,816 - INFO - Processing file name: WhatsApp Chat with Albert Phiri Edoofa (16_11) SZ.txt
2023-11-29 16:30:57,816 - INFO - Processing file name: WhatsApp Chat with Blessing Mbewe Edoofa (24_11) SZ.txt
2023-11-29 16:30:57,816 - INFO - Processing file name: WhatsApp Chat with Blessing Mbewe Edoofa (24_11) SZ.txt
2023-11-29 16:30:57,816 - INFO - Processing file name: WhatsApp Chat with Blessing Mbewe Edoofa (24_11) SZ.txt
2023-11-29 16:30:57,816 - I

2023-11-29 16:30:57,845 - INFO - Processing file name: WhatsApp Chat with Lynette Nyasha Kadzuma Edoofa (20_11) SZ.txt
2023-11-29 16:30:57,845 - INFO - Processing file name: WhatsApp Chat with Lynette Nyasha Kadzuma Edoofa (20_11) SZ.txt
2023-11-29 16:30:57,845 - INFO - Processing file name: WhatsApp Chat with Lynette Nyasha Kadzuma Edoofa (20_11) SZ.txt
2023-11-29 16:30:57,845 - INFO - Processing file name: WhatsApp Chat with Lynette Nyasha Kadzuma Edoofa (20_11) SZ.txt
2023-11-29 16:30:57,845 - INFO - Processing file name: WhatsApp Chat with Malvern Chiwashira Edoofa (14_11) SZ.txt
2023-11-29 16:30:57,851 - INFO - Processing file name: WhatsApp Chat with Malvern Chiwashira Edoofa (14_11) SZ.txt
2023-11-29 16:30:57,851 - INFO - Processing file name: WhatsApp Chat with Mapfumo Tinotenda Edoofa (10_11) SZ.txt
2023-11-29 16:30:57,851 - INFO - Processing file name: WhatsApp Chat with Mapfumo Tinotenda Edoofa (10_11) SZ.txt
2023-11-29 16:30:57,852 - INFO - Processing file name: WhatsApp Ch

2023-11-29 16:30:57,872 - INFO - Processing file name: WhatsApp Chat with Stanley Maveneke Edoofa (22_11) SZ.txt
2023-11-29 16:30:57,872 - INFO - Processing file name: WhatsApp Chat with Tabeth Mandembwe Edoofa (24_11) SZ.txt
2023-11-29 16:30:57,879 - INFO - Processing file name: WhatsApp Chat with Tabeth Mandembwe Edoofa (24_11) SZ.txt
2023-11-29 16:30:57,879 - INFO - Processing file name: WhatsApp Chat with Tabeth Mandembwe Edoofa (24_11) SZ.txt
2023-11-29 16:30:57,879 - INFO - Processing file name: WhatsApp Chat with Tafara Mukucha Edoofa (23_11) SZ.txt
2023-11-29 16:30:57,879 - INFO - Processing file name: WhatsApp Chat with Tafara Mukucha Edoofa (23_11) SZ.txt
2023-11-29 16:30:57,879 - INFO - Processing file name: WhatsApp Chat with Takudzwa Chari Edoofa (27_11) SZ.txt
2023-11-29 16:30:57,879 - INFO - Processing file name: WhatsApp Chat with Takudzwa Chinyama Edoofa (22_11) SZ.txt
2023-11-29 16:30:57,879 - INFO - Processing file name: WhatsApp Chat with Takudzwa Chinyama Edoofa (2

                 Date   Person           Chat Group Name  Delay  \
0 2023-11-27 18:39:00  Arshita     Abdul_Edoofa(28_11)AA   True   
1 2023-11-27 19:22:00  Arshita   Alinafe_Edoofa(14_11)AA   True   
2 2023-11-27 12:09:00  Arshita  Alistair_Edoofa(07_11)AA   True   
3 2023-11-27 12:28:00  Arshita  Alistair_Edoofa(07_11)AA   True   
4 2023-11-27 19:25:00  Arshita     Angel_Edoofa(16_11)AA   True   

   Delay in Mins                                 Reference Messages  
0          100.0  [27/11/23, 4:49 pm - Arshita added you\n, 27/1...  
1          413.0  [ *A problem is a chance for you to do your be...  
2           74.0  [\n, How are you doing?\n, 27/11/23, 12:01 pm ...  
3           19.0  [●Physics &\n, ●Technical Graphics & Design \n...  
4          415.0  [ *A problem is a chance for you to do your be...  


In [18]:
delay_analysis_df.to_csv("delays.csv", index=False)

In [ ]:
all_delay_data